In [1]:
import pandas as pd
import ast
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import altair as alt
from sklearn.manifold import TSNE
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from pyvis.network import Network

In [4]:
df_path = 'elasticsearch_data.csv'
data = pd.read_csv(df_path, sep=',')

In [5]:
# Function to combine and remove duplicate tags
def combine_tags(row):
    all_tags = []
    
    # Add tags from 'category' column
    all_tags.extend(eval(row['category']))
    
    # Add tags from 'title_categories' column (if not empty)
    if pd.notna(row['title_categories']):
        all_tags.extend(eval(row['title_categories']))
    
    # Add tags from 'summarize_categories' column (if not empty)
    if pd.notna(row['summarize_categories']):
        all_tags.extend(eval(row['summarize_categories']))
    
    # Remove duplicates and return as a comma-separated string
    unique_tags = list(set(all_tags))
    return unique_tags


# Create a new column with combined and unique tags
data['category'] = data.apply(combine_tags, axis=1)
data.drop(['title_categories','summarize_categories'], axis=1)
data.head()

,category,title_categories,summarize_categories,title,author,fascicle,year,start_page,end_page,pdf_link,id
0,"[Vida, Filosofía, Filosofía y Política, Políti...",['Determinismo'],"['Determinismo', 'Vida', 'Variedad', 'Sucesión...",El determinismo,Juan Zaragüeta Bengoechea,1,1973,9,20,abrir_pdf.php?id=ANU-M-1973-10000900020,ANU-M-1973-10000900020
1,"[Vida, Convivencia, Filosofía, Filosofía y Pol...","['Ciudades', 'Formas', 'Convivencia']","['Ciudad', 'Ciudades', 'Vida', 'Social', 'Pobl...",Las nuevas ciudades: hacia nuevas formas de co...,Manuel Fraga Iribarne,1,1973,21,51,abrir_pdf.php?id=ANU-M-1973-10002100051,ANU-M-1973-10002100051
2,"[Gobierno, Anarquía, Aggiornamento, Filosofía,...","['Aggiornamento', 'Clasificación', 'Formas', '...","['Democracia', 'Forma', 'Pueblo', 'Anarquía', ...","""Aggiornamento"" de la clasificación de las for...",Antonio Perpiñá Rodríguez,1,1973,53,71,abrir_pdf.php?id=ANU-M-1973-10005300071,ANU-M-1973-10005300071
3,"[Consejo, Formulaciones, Gobierno, Filosofía, ...",['Formulaciones'],"['Consejo', 'Cortes', 'Gobierno', 'Leyes']",Las formulaciones constitucionales,José María Cordero Torres,1,1973,73,108,abrir_pdf.php?id=ANU-M-1973-10007300108,ANU-M-1973-10007300108
4,"[Desarrollo, Sociología, Idea, Economía, Evolu...","['Progreso', 'Crisis', 'Cambio']","['Historia', 'Desarrollo', 'Idea', 'Evolución'...",Progreso y crisis en el cambio histórico,Carlos Ruiz del Castillo,1,1973,111,128,abrir_pdf.php?id=ANU-M-1973-10011100128,ANU-M-1973-10011100128


In [7]:
# Create an empty DataFrame for the new data
new_data = pd.DataFrame(columns=['source', 'target'])

# Iterate through each row in the existing DataFrame
for index, row in data.iterrows():
    categories = row['category']
    title = row['title']
    # Add a row for each individual category
    for category in categories:
        new_data = new_data.append({'source': title, 'target': category}, ignore_index=True)

# Save the new data to a CSV file
new_data.to_csv('all_categories_target.csv', index=False)

In [1]:
import pandas as pd
from pyvis.network import Network

# Load the relations dataset
# relations_df = pd.read_csv('all_categories_target.csv')

# Count the frequency of each target node
node_counts = relations_df['target'].value_counts().reset_index()
node_counts.columns = ['target', 'frequency']

# Create a PyVis network object
net = Network(notebook=True, width=1000, height=600)

# Add target nodes with sizes based on frequency
for _, row in node_counts.iterrows():
    category = row['target']
    frequency = row['frequency']
    net.add_node(category, size=frequency)

# Create edges between target nodes with the same source
for source, group in relations_df.groupby('source'):
    target_nodes = list(group['target'])
    if len(target_nodes) > 1:
        for i in range(len(target_nodes)):
            for j in range(i + 1, len(target_nodes)):
                net.add_edge(target_nodes[i], target_nodes[j])

# Show the network graph
net.show('target_node_network_graph.html')

target_node_network_graph.html
